# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
try:
    from api_keys import weather_api_key  # Option 1: import from api_keys.py
except ModuleNotFoundError:
    # Option 2: fallback to direct assignment if api_keys.py not found
    weather_api_key = "51646574f1d475b83687474ef612a846"
    


In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,14.87,67,0,3.63,AU,1746630983
1,1,punta arenas,-53.1500,-70.9167,3.06,75,100,4.63,CL,1746630984
2,2,port-aux-francais,-49.3500,70.2167,4.41,92,100,2.56,TF,1746630986
3,3,padang,-0.9492,100.3543,26.52,90,56,1.11,ID,1746630988
4,4,thompson,55.7435,-97.8558,7.09,45,40,5.66,CA,1746630989


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
56,56,cabo san lucas,22.8909,-109.9124,22.17,56,0,2.06,MX,1746631045
59,59,hamilton,39.1834,-84.5333,21.11,66,0,2.57,US,1746630828
122,122,hotan,37.0997,79.9269,26.98,11,0,4.17,CN,1746631119
129,129,carnarvon,-24.8667,113.6333,26.87,30,0,4.27,AU,1746631127
133,133,chibuto,-24.6867,33.5306,24.18,46,0,0.31,MZ,1746631131
167,167,birzebbuga,35.8258,14.5269,24.38,53,0,2.57,MT,1746631169
295,295,richards bay,-28.7830,32.0377,23.16,63,0,3.81,ZA,1746631309
318,318,mahina,-17.5065,-149.4890,23.87,83,0,1.03,PF,1746631334
324,324,kailua-kona,19.6406,-155.9956,22.53,82,0,2.06,US,1746631342
331,331,westport,41.1415,-73.3579,21.20,66,0,4.12,US,1746631327


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
56,cabo san lucas,MX,22.8909,-109.9124,56,
59,hamilton,US,39.1834,-84.5333,66,
122,hotan,CN,37.0997,79.9269,11,
129,carnarvon,AU,-24.8667,113.6333,30,
133,chibuto,MZ,-24.6867,33.5306,46,
167,birzebbuga,MT,35.8258,14.5269,53,
295,richards bay,ZA,-28.7830,32.0377,63,
318,mahina,PF,-17.5065,-149.4890,83,
324,kailua-kona,US,19.6406,-155.9956,82,
331,westport,US,41.1415,-73.3579,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
import requests

# Make sure to define your API key
geoapify_key = "e896d1cc128743b289ed1ae4de19b8d0"  

# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    try:
        # Make an API request using the params dictionary
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # raise exception if request failed

        # Convert the API response to JSON format
        data = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        hotel_name = data["features"][0]["properties"]["name"] if data["features"] else "No hotel found"

    except (requests.RequestException, KeyError, IndexError):
        # If API request fails or no hotel found
        hotel_name = "No hotel found"

    # Save hotel name into DataFrame
    hotel_df.at[index, "Hotel Name"] = hotel_name

    # Log the search results
    print(f"{row['City']} - nearest hotel: {hotel_name}")

# Display sample data
hotel_df.head()



Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
hamilton - nearest hotel: North Vista Manor
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
carnarvon - nearest hotel: No hotel found
chibuto - nearest hotel: DESHENG
birzebbuga - nearest hotel: Seabreeze
richards bay - nearest hotel: SUN1 Richards Bay
mahina - nearest hotel: Motu Martin
kailua-kona - nearest hotel: PACIFIC 19 Kona
westport - nearest hotel: Norwalk Inn and Conference Center
xai-xai - nearest hotel: No hotel found
greenville - nearest hotel: Quality Inn
san angelo - nearest hotel: Hilton Hotel
port saint john's - nearest hotel: Outback Inn
ixtapa - nearest hotel: Ma. Cristina
mthatha - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
56,cabo san lucas,MX,22.8909,-109.9124,56,Comfort Rooms
59,hamilton,US,39.1834,-84.5333,66,North Vista Manor
122,hotan,CN,37.0997,79.9269,11,Hua Yi International Hotel (accepts foreigners)
129,carnarvon,AU,-24.8667,113.6333,30,No hotel found
133,chibuto,MZ,-24.6867,33.5306,46,DESHENG


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)